In [1]:
from dotenv import load_dotenv
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr
from kaggle_secrets import UserSecretsClient

import kagglehub
from transformers import AutoTokenizer, AutoModelForCausalLM
import logging, warnings
logging.getLogger("torch").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

# 🧰 Tool Utilization

### 📱 Introducing Pushover

**Pushover** is an efficient and reliable tool for delivering **real-time push notifications** directly to your mobile device, tablet, or desktop.  
It enables seamless communication between your applications and your devices through a simple API.

---

### ⚙️ Setup Guide

#### 1. Account Registration
Visit [Pushover.net](https://pushover.net/) and select **“Login or Signup”** in the upper-right corner to create your free account.

#### 2. Create an Application Token
Once logged in:
- Navigate to your Pushover home screen.
- Click **“Create an Application/API Token.”**
- Assign any preferred name (for example, `Agents`) and select **“Create Application.”**

#### 3. Configure Environment Variables
Add your credentials to the `.env` file in your project directory:



In [2]:
user_secrets = UserSecretsClient()
pushover_token= user_secrets.get_secret("PUSHOVER_TOKEN")
pushover_user = user_secrets.get_secret("PUSHOVER_USER")

pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


# Send Push Message to Mobile

In [3]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

push("hello i am sending message from Kaggle")

Push: hello i am sending message from Kaggle


In [4]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}


def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [5]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [6]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [7]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [8]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'The email address of this user'},
     'name': {'type': 'string',
      'description': "The user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional information about the conversation that's worth recording to give context"}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['quest

## 🧠 Handling Tool Calls in the Agent Workflow

The following function, `handle_tool_calls()`, is responsible for managing and executing a list of tool calls received by the system.  
Each tool call specifies which function to invoke and provides the necessary arguments in JSON format.

---

### 🔍 Function Overview

- **Input:**  
  A list of tool call objects, each containing:
  - `function.name` → The name of the tool or function to execute.  
  - `function.arguments` → JSON-formatted arguments for that function.  

- **Process:**  
  The function iterates over each tool call and uses a **conditional statement (the "Big IF Statement")** to determine which function to execute.

- **Output:**  
  A list of results, each containing:
  - The role (`"tool"`)
  - The JSON-encoded result of the function call
  - The corresponding `tool_call_id` for tracking

---

### 🧩 Key Logic — The “Big IF Statement”

Within the function, the `if-elif` block acts as the **decision layer**:

- `record_user_details()` → Captures and stores user information.  
- `record_unknown_question()` → Logs unrecognized or unsupported queries for later review.

Each executed tool call’s result is then appended to the `results` list, forming a complete response package.


In [9]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [10]:
globals()["record_unknown_question"]("How can I optimize this model’s hyperparameters automatically?")


Push: Recording How can I optimize this model’s hyperparameters automatically? asked that I couldn't answer


{'recorded': 'ok'}

## ⚙️ A More Elegant Approach — Dynamic Tool Handling

This version of `handle_tool_calls()` improves on the previous design by eliminating the long chain of conditional statements.  
Instead, it dynamically resolves which function to execute based on the tool’s name.

---

### 🧩 How It Works

- **Input:** A list of tool call objects, each containing a tool name and its corresponding arguments.  
- **Dynamic Resolution:** The function looks up the tool in the global namespace at runtime. If a matching function is found, it is executed with the provided arguments.  
- **Safe Handling:** If the tool name does not correspond to any known function, the call safely returns an empty result.  
- **Output:** A list of structured results, including the tool’s output and the associated tool call identifier.

---

### 💡 Advantages

- **Scalable:** Adding new tools requires no changes to the handler function.  
- **Concise:** Removes repetitive conditional logic, improving readability and maintainability.  
- **Dynamic Execution:** Supports runtime function invocation based on tool names, making it flexible and adaptable.



In [11]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [12]:
from pypdf import PdfReader

pdf_path = "/kaggle/input/custom-image-dataset/Profile.pdf"
reader = PdfReader(pdf_path)

extracted_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        extracted_text += text + "\n"

summary_path = "/kaggle/working/summary.txt"
with open(summary_path, "w", encoding="utf-8") as f:
    f.write(extracted_text)

print(f"PDF successfully converted and saved to {summary_path}")

PDF successfully converted and saved to /kaggle/working/summary.txt


In [13]:
reader = PdfReader("/kaggle/input/custom-image-dataset/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("/kaggle/working/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

In [14]:
system_prompt = f"""
You are now acting as {name}, representing {name}'s professional persona on their website. 
Your role is to answer questions regarding {name}'s career, background, skills, and professional experience. 
Always respond in a professional, engaging, and approachable manner, as if speaking to a potential client, collaborator, or future employer.

You are provided with a comprehensive summary of {name}'s background and LinkedIn profile. Use this information to provide accurate and helpful answers. 

If you encounter a question that you cannot answer, regardless of its relevance or complexity, use the `record_unknown_question` tool to log it. 
If the user provides contact information, especially their email, encourage them to share it and record it using the `record_user_details` tool.

Your responses should guide conversations naturally toward meaningful engagement, including professional opportunities, collaborations, or further contact via email. 
Maintain consistency in tone, professionalism, and accuracy at all times.

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

With this context, engage with the user in character as {name}, providing informative, professional, and helpful responses.
"""


In [15]:
from google import genai
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
gemini_api_key = user_secrets.get_secret("GEMINI_API_KEY")

client = genai.Client(api_key=gemini_api_key)

def chat(message, history):
    prompt = "\n".join(
        [system_prompt] +
        [f"User: {h['content']}" if h["role"] == "user" else f"{name}: {h['content']}" for h in history] +
        [f"User: {message}", f"{name}:"]
    )

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    reply = response.text.strip()

    if "record_unknown_question" in reply:
        record_unknown_question(message)
        reply = reply.replace("record_unknown_question", "").strip()
    if "record_user_details" in reply:
        record_user_details(message)
        reply = reply.replace("record_user_details", "").strip()

    return reply


In [16]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://994f236ae129f53a0a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
